In [ ]:
# Importing libraries for Named Entity Recognition 
import spacy
nlp=spacy.load('en_core_web_sm')
# Training Spacy model
from spacy.training.example import Example
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# Getting the pipeline component
ner=nlp.get_pipe("ner")

import pandas as pd

# Importing libraries for string matching
import re

# Training

In [ ]:
# Training data based on few outputs and misidentifcation.
TRAIN_DATA = [
              ("GANOHIMARG is a leading e-commerce place", {"entities": [(0, 9, "GPE")]}),
              ("I reached GANOHIMARG yesterday.", {"entities": [(11, 20, "GPE")]}),
              ("I reached DISTRICT GAUTAMBUDH NAGAR yesterday.", {"entities": [(10, 35, "GPE")]}),
              ("I reached LUCKNOW yesterday.", {"entities": [(11, 18, "GPE")]}),
              ("I went to UTTAR PRADESH.", {"entities": [(11, 23, "GPE")]}),
              ("I went to NOIDA, UTTAR PRADESH.", {"entities": [(11, 23, "GPE")]}),
              ("I went to UTTAR PRADESH WEST", {"entities": [(11, 23, "GPE")]}),
              ("I went to CHENNAI.", {"entities": [(11, 17, "GPE")]}),
              ("I went to JEEVAN TARA BUILDING.", {"entities": [(11, 29, "GPE")]}),
              ("I went to Jeevan Tara Bulg.", {"entities": [(11, 26, "GPE")]}),
              ("I went to Jeevan Tara Building.", {"entities": [(11, 29, "GPE")]}),
              ("I recently ordered a book from Amazon", {"entities": [(24,32, "ORG")]}),
              ("I ordered this from 226016", {"entities": [(20,26, "PINCODE")]}),
              ("Sth day of May 2022 is a date of trial. ", {"entities": [(0,19, "DATE")]}),
              ("12th day of May 2022 is a date of trial. ", {"entities": [(0,19, "DATE")]}),
              ("01.8.2022 is the wedding date.",{"entities": [(0,8, "DATE")]}),
              ("29-7-2022 is the joining date.",{"entities": [(0,8, "DATE")]}),
              ("29-07-2022 is the joining date.",{"entities": [(0,8, "DATE")]}),
              ("3/8/2022 is the ceremony date.",{"entities": [(0,8, "DATE")]}),
              ("23/11/2022 is the joining date.",{"entities": [(0,8, "DATE")]}),
              ("03/08/2022 is the ceremony date.",{"entities": [(0,8, "DATE")]}),
              ("01.08.2022 is the wedding date.",{"entities": [(0,8, "DATE")]}),
              ("I rented a camera", {"entities": [(12,18, "PRODUCT")]}),
              ("I came at 10:30AM", {"entities": [(11,18, "TIME")]}),
              ("I came at 4:35PM", {"entities": [(11,18, "TIME")]}),
              ("I came at 02:50PM", {"entities": [(11,18, "TIME")]}),
              ("I came at 9:18AM", {"entities": [(11,18, "TIME")]}),
              ("I came at 4.30AM", {"entities": [(11,18, "TIME")]}),
              ("I came at 7.40PM", {"entities": [(11,18, "TIME")]}),
              ("I came at time 10.30", {"entities": [(11,16, "TIME")]}),
              ("The time is 10.30", {"entities": [(12,17, "TIME")]}),
              ("The time is 9.30", {"entities": [(12,16, "TIME")]}),
              ("I met Chandra Goel", {"entities": [(7,18, "PERSON")]}),
              ("I met Mr. Mithilesh Pathak", {"entities": [(6,26, "PERSON")]}),
              ("I met Yash Satam", {"entities": [(7,18, "PERSON")]}),
              ("25,00,000 were paid",{"entities": [(0,8, "MONEY")]}),
              ("Rs 45,00,000 were paid",{"entities": [(0,11, "MONEY")]}),
              ("11,00,000 Rs were paid",{"entities": [(0,11, "MONEY")]}),
              ("Thirty Five Lacs Only were paid",{"entities": [(0,21, "MONEY")]}),
              ("Thirty Five Lacs Only were paid",{"entities": [(0,16, "MONEY")]}),
              ("Fifty Seven Lakhs Only were paid",{"entities": [(0,22, "MONEY")]}),
              ("Fifty Seven Lakhs Only were paid",{"entities": [(0,17, "MONEY")]}),
              ("Sixty Seven Tousand Only were paid",{"entities": [(0,24, "MONEY")]}),
              ("Sixty Seven Tousand Only were paid",{"entities": [(0,19, "MONEY")]}),
              ("Cost of this is Rs 3000",{"entities": [(16,23, "MONEY")]}),
              ("Cost of this is 4,000",{"entities": [(16,21, "MONEY")]}),
              ("Case Id is Tribunal 2305-2022", {"entities": [(11,27, "CASE_ID")]}),
              ("Case Id is Tribunal 2305-2022", {"entities": [(11,19, "TRIBUNAL")]}),
              ("Case Id from DEBTS RECOVERY TRIBUNAL", {"entities": [(13,36, "TRIBUNAL")]}),
              ("Case Id from Debts Recovery Tribunal", {"entities": [(13,36, "TRIBUNAL")]}),
              ("KOTAK MAHINDRA BANK is my bank.", {"entities": [(0,19, "ORG")]}),
              ("HINDON RIVER MILLS LT is my company.", {"entities": [(0,21, "ORG")]}),
              ("Kotak Mahindra Bank is my bank.", {"entities": [(0,19, "ORG")]}),
              ("Hindon River Mills LT  is my company.", {"entities": [(0,21, "ORG")]}),
              ("Si Prem Prakash Nayar & Ors is my company.", {"entities": [(0,27, "ORG")]}),
              ("110001 is an area", {"entities": [(0,7, "PINCODE")]}),
              ("2608 is an area", {"entities": [(0,5, "PINCODE")]}),
              ("I got a call from 23748468", {"entities": [(18,27, "PHONE")]}),
              ("9999122470 is my number.", {"entities": [(0,11, "PHONE")]}),
              ("You can call me on 25206819 in case of any emergency.", {"entities": [(20,29, "PHONE")]}),
              ("STARKER AUTOMATION PRIVATE LIMITED is recognized as leader in market",{"entities": [(0,34, "ORG")]}),
              ("I came from ATUL GORVER MARG", {"entities": [(12,28, "GPE")]})
              ]

In [ ]:
import re
test_str = "Si Prem Prakash Nayar & Ors is my company."
for match in re.finditer('Si Prem Prakash Nayar & Ors', test_str):
  print(match.start(), match.end())

0 27


In [ ]:
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [ ]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 50 iterations
  for iteration in range(50):

    # shuufling examples before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example=[]
        for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "I reached LUCKNOW yesterday." with entities "[(11, 18, 'GPE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "2608 is an area" with entities "[(0, 5, 'PINCODE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "I came at 4:35P

# Text Data

In [ ]:
dec_8_2021 = nlp('''
OFFICE OF THE RECOVERY OFFICER.I, DEBTS RECOVERY TRIBUNAL I, DELH! Meant
4th FLOOR, JEEVAN TARA BUILDING, PARLIAMENT STREET, NEW DELHL-110001 (MSMELIMtuos

RC no. 022018 Dated SORE
KOTAK MAHINDRA BANK VS HINDON RIVER MILLS LT.

‘As pr my oder dete 30.08,2022 the under mentioned properties wl be sol by publ action sale on 17,.062022in he sid
Recovery Cartiale

The auction sale willbe “online e- Auctioning” through website

hitosldetauctiontinernt
Date and Time of Auction: 17.6.2002 between 03.00 pm an 04.00 pm (with extensions of minutes duration
‘afer 04:00 PM required)

‘Deszripton of Prepary Fares ana Oepo5t (END) THR]
ergaed and (ocrwasving S26 aed 21 ZB hectares, alse | — 1360000000 3.50000

titans manda an rate senate aed & NH 24

Bana hasisod Nea Dana ToL East Pritera Expressway Ds
(Ghazatad UP owned by Hedon Rr Mi

The actonsale wilt ‘online hrughe-auchonfreug wetste porta: higsidLautiontigez net

2. The END shale paid either through Demand DrafvPay Order favour ofthe “Recovery Office ORT, Deli-Ale RC.
‘No, 1272018" or through NEFTIRTCS to the in Favour of Kotak Mahindra Bank, Account No. 6411307138, IFSC Code
‘KKK0000646 of Kotak Mahindra Bank Lined, Malad (East) Mum, Branch. Tho sid Demand DrtiPay Ode |
cigial poo of payment tough NEFTIRTGS qia EMD, alongwith se atested copy of ently (oles cad J ering
Iconsefpasspot, which shoud onan Be adress fue communication and sa ats cop of PAN car, adn he cae
of company, copy of reson passed by the board menbars of te Conpary or any cher document confining
reprsentaton/atiomoy of he conpany alo, must reach th offie of the Recovery Officer, ORTH, Oth [atest by
‘1506-202 before 4.00PM, The EMD or org prof of EMD received heeatshal nobeconsderes

3, The envetepeconaring EMD or oil poo of payment of EMD throug NEFTIRTCS, alngath dtals of he sender, ie
‘tress emallD and moi umber ete. shoud be buper sorbed "RC No 022018",

4, Thepropeisteingsokion“asis where is andasis whats bass”

5. Theirs ae advised ogo trough he poral tsi autiontige ne for ete tems and condtion foe aucon sale
bor suiting fei bs and tking par inthe eauctn sle proceedings and fr cota! Sh, Mano] Chauhan, Senior

Manager MebileNo.9673179687,Emall;manod kchauhan@ketakcom,

5. espe ders erst reps races whe pt en tan Dat Passed a isaac,
epost of EMD, which s manda for iéing he shove e-acton, fom Mis. E-Procu nologies Lt
ace Te Abmibd 6704708, Wa See Onset —Or Cb. Mu mt Coleg. Ee ree

8. Oni os ters noing vad er D8 Passwords an cred payment EMO tough comand da yO or NEFT
[RTGS calbeelgbletopartcpateinon ne e-aucton shale elle parpate non nee auction,

‘The Prospective buyemay inspect 0 01.8.2022 & 02.06.2072 rom 10:30AM 004.0 PL

10 The iret isrs ho have submited habs ol Below the raerve price by 15.06.2022 in the offce of the Recovery
Offce, DRT, Det. Sale elgble participate ine-auctonta be hel on fom 03 MPM to 0.00PM on 17.08.2022 incase
‘dis plod heat minutes ofthe desing tine ct he aucon, th cosing ere wil automaticaly el extended fr Sines,

11. The bidders shalerrove tex ofrin mils oR. 25, 0,000-Rupees Twenty five Lacs only)

12 The unsuccessful bier hata he END direct ome Office of Recovery Officer, ORT, Delhi CH Bank Le. Kotak
Mahindra Bank nod on Cosi hee aucton Sale Proceedings.

13, The svcasi/hghet bier shalhae prepare and dpast Domand Dra Pay cde for 25% he id sale amount ate |
‘ating the EMD Faverng Recovery Oe DR eh AC RC.NO. 1022018 by nertbank Working day. by 0:90PM
wi his tba fing whichhe EMD shale freed.

14 Thesuecesstl highest der sal be depst, rough eran Dr pay onder favring Recovery ofoe.I, ORT, Dei A
.C.NO.1022018, th balance 75% of he sae Proceeds before he Recovery Ofer -I, DRT Deion or etre 15th day tor
thedateofaucion ofthe propery exusive of such day, orth 15h daybe Suny oroher hola, hen one stoic day
tere 151h dy. ong wih te poundage fee @2% upto R600 2nd@Qt%s on he excess of such gress araunt ov
s.100tinfvorofRegstar DRT Dei (ncaseofdepostof balance ancunt of 75% Though post the same shoul reach
‘he Recovery fier as above), In Case of default payment within he prescribed period he propery shaders, ai he
Issue of fesh proclamation of ae. The depos, alr detayng he expenses ofthe se, may, ithe undersigned his fi be
ferfoaato be Govemmontand be delauingpuchaser shal foetal cam Be propery re ary parole sum rhe
imayecheaguniybostt

''')

text=[]
label = []

for ent in dec_8_2021.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_dec_8_2021= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_dec_8_2021

,Text,Label
0,DEBTS RECOVERY TRIBUNAL,TRIBUNAL
1,4th FLOOR,GPE
2,JEEVAN TARA BUILDING,GPE
3,MSMELIMtuos\n\nRC,TRIBUNAL
4,KOTAK MAHINDRA BANK VS HINDON RIVER MILLS LT,ORG
5,"30.08,2022",DATE
6,Recovery Cartiale,ORG
7,03.00,TIME
8,04.00,TIME
9,04:00,TIME


In [ ]:
# 3 Dec 2021
dec_3_2021 = nlp('''
‘600, University Road, Near Hanuman Setu Temple, Lucknow-226007
(Area of Jurisdiction: Pat of Uttar Pradesh)
Summons for filling Reply and Appearance by Publication
‘Summons to defendants under Section 19(3) ofthe Recovery of Debts and Bankruptcy
‘Act, 1983 read with Rules 12 and 13 ofthe Debts Recovery Tribunal (Procedure) Rules,

1998,

Original Application No. 571/2020 DATE: 28.10.2021
Canara Bank RAM Gaba
VS
To, Si Prem Prakash Nayar & Ors,
4-PramPrkash Nayar SioRa Kuma Nya RloHouse No. 1219, Boa A Fest loo Jahangr Pus
New Da 10033, ALSO AT: '-57H0, Grou Fer, Jahangi Pur NS. Mar, NarsWest Deh,
Debs 1103,
2 Any Kura Naya Slo Ra Kura Nayar, Rl House No 1219, Bled, Ft Flor, Jang Ps
Nena: Ho ALSOAE K-70, Grid, PURNS Mand Nee
ett
$-Mis Morpheus Devdopers Pata Lita, Restored fice A St Manta Gk, Map Oa
1069 troughs auPrerzed pny.
Inthe above noted aplcatn, you are aquired fle repyn Paper Book fom nt sts ong with
eran ads ar poral ee ody actors oer ee
Ss Tana afi serving copy of sane on fe apleart oris counsel dl
Discs ote Suns td cle open three iburalon 220 EDA,
{ainguhichine spleatn shale heard ad decided in your absence,

Registrar; Debts Recovery Tribunal, Lucknow.

‘600, University Road, Near Hanuman Setu Temple, Lucknow-226007
(Area of Jurisdiction: Pat of Uttar Pradesh)
Summons for filling Reply and Appearance by Publication
‘Summons to defendants under Section 19(3) ofthe Recovery of Debts and Bankruptcy
‘Act, 1983 read with Rules 12 and 13 ofthe Debts Recovery Tribunal (Procedure) Rules,

1998,

Original Application No. 571/2020 DATE: 28.10.2021
Canara Bank RAM Gaba

To, Si Prem Prakash Nayar & Ors,
4-PramPrkash Nayar SioRa Kuma Nya RloHouse No. 1219, Boa A Fest loo Jahangr Pus
New Da 10033, ALSO AT: '-57H0, Grou Fer, Jahangi Pur NS. Mar, NarsWest Deh,
Debs 1103,
2 Any Kura Naya Slo Ra Kura Nayar, Rl House No 1219, Bled, Ft Flor, Jang Ps
Nena: Ho ALSOAE K-70, Grid, PURNS Mand Nee
ett
$-Mis Morpheus Devdopers Pata Lita, Restored fice A St Manta Gk, Map Oa
1069 troughs auPrerzed pny.
Inthe above noted aplcatn, you are aquired fle repyn Paper Book fom nt sts ong with
eran ads ar poral ee ody actors oer ee
Ss Tana afi serving copy of sane on fe apleart oris counsel dl
Discs ote Suns td cle open three iburalon 220 EDA,
{ainguhichine spleatn shale heard ad decided in your absence,

Registrar; Debts Recovery Tribunal, Lucknow.
''')

text=[]
label = []

for ent in dec_3_2021.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_dec_3_2021= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_dec_3_2021

,Text,Label
0,University Road,ORG
1,Uttar Pradesh,ORG
2,Rules 12,MONEY
3,13,MONEY
4,Debts Recovery Tribunal,TRIBUNAL
5,28.10.2021,TIME
6,Canara Bank RAM Gaba,ORG
7,Si Prem Prakash Nayar & Ors,ORG
8,4-PramPrkash Nayar SioRa Kuma Nya RloHouse,MONEY
9,Boa A Fest,ORG


In [ ]:
# 20 May 2022 delhi
may_20_2022 = nlp('''
BEFORE DEBTS RECOVERY TRIBUNAL-III, DELHI
‘4th Floor Jeevan Tara Building Parliament Street, New

Notice under Secion 19 (4) of he Recovery of Debts Due fo Banks and Fanci
Insttors At 1985 ead wih Rue 128 13 oe Debs Recovery Taba Prooeire
Rules) 1983).

In the matter of

OA No. 8112021

IDFC FIRST BANK LTD. Vs STARKER AUTOMATION PRIVATE LIMITED AND ORS |
‘plcant Detendants

‘STARKER AUTOMATION PRIVATE LIMITED
SRO FLOOR, FLAT NO. 31, SURYA KIRAN BUILONNG, 19 KASTURBA
1b | GANOHIMARG, CONNAUGHT PLACE, NEW DELM-on01

ALSOAT:
‘i, TELEGRAPH LANE, ATUL GORVER MARG, CP. NEW DELME-1t0001
ALSO:
1582, RING ROAD NDRA GANDMY NAGAR, LUCKNOW UTTAR PRADESH 226016
‘WANOJ KUMAR.
SRO FLOOR, FLAT NO. 31, SURYA KIRAN BUILONNG, 19 KASTURBA
‘GANDHI MARG, CONNAUGHT PLACE, NEW DELHL 110001
22 | aLsoat:

HIM, TELEGRAPH LANE, ATUL GORVER MARG, CP NEW DEL: 110001
ALSOAT
1882, RING ROAD INDRA GANDMI NAGAR, LUCKNOW UTTAR PRADESH 2018
‘SHASHITRVEDL
SHO FLOOR, FLAT NO. 31, SURYAKIRAN BUILDING, 19 KASTURBA
‘GANOHi MARG, CONNAUGHT PLACE, NEW DELHLti000t
ALSOAT:
‘ibd, TELEGRAPH LANE, ATUL GORVER MARG, CP. NEW DELME110001
ALSOAT:
1582 RING ROAD NORA GANOHE NAGAR, LUCKNOW, UTTAR PRADESH 2608]

Wnereas the above named applicant has insted a case agains! you and whereas thas
been shown othe satisfaction of he Tnbuna! hats not possible fo Serve youn ori
way hereto. hs noc sven by advertsoment recting you to make appearance |
tes Tribunal 2305-2022 t 10:30AM
Take notice that in case of you faze to appear onthe above mentioned day bao Bis |
Tribunal, te casewillbeheard and decded n your absence
Due to ongoing Pandemic stuaton, al the matters wil be taken up tough Video|
Conferencing and frat purpose
() Al the Advocatesttigants shall dowioad the Cisco Webex appiation 'sware
fa) Meetng IO and Password forthe next date of hearing qua cases to be taken by|
RegistrarRecovery Oftcer-Vand Recovery Oficer. shal be avaiate one day rr
{othenextdateatORT Ofical Portal. govinunder the Pubic Notce Hees
(a) Inany exigency qua that, the Advocates/iigans can contact he concemed oil at |
Ph No, 23748468
‘Given undermy hand and eal othis Tibunalon this the Sth day of May 2022

By Order of the Tribunal
''')

text=[]
label = []

for ent in may_20_2022.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_may_20_2022= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_may_20_2022

,Text,Label
0,BEFORE DEBTS RECOVERY TRIBUNAL-III,ORG
1,Floor Jeevan Tara Building Parliament Street,ORG
2,New\n\nNotice,GPE
3,Debts Due fo,TRIBUNAL
4,1985,DATE
5,13,DATE
6,Debs Recovery Taba Prooeire,TRIBUNAL
7,8112021,PINCODE
8,IDFC FIRST BANK LTD,ORG
9,Vs STARKER AUTOMATION PRIVATE LIMITED,ORG


In [ ]:
# 7 July 2022
july_7_2022 = nlp('''
[Petore Debts Recovery Tribunal, Oe
“Foo, Jeevan Tra Bulg,
Parma Sra. New On 10001
Tetce under section 19(4) of the
Recovery of Debts Ove to Banks nd
Francia tutions At, 1983 ead wth
[Rule 12 413 of tho Dns Rocovr|

tural recor Rules) 193

ine mater of OA Wo. 322027

[PUNJAB NATIONAL BANK
‘opin

i"
ay PRASAD AND ORS.

He
+s Way Prana, Ro 28909, Est En
aprons, Wayur Visa Phase-1
Exar, New D108,

10-2 Ws Sia Prasad (Jit Appt)
io 2808 East Ent Aganors May
ea Paso, Exmnsen. Now Od
rots

0-3 Ws Mophaus Developers Pvt. Li
og ce Hous No 1, SMa al
at asp Oath 10053 Ano a 14
Feo: Tower, To Cot To
Pc Na. A, Soc #2, Nad 20130)
uP.

rms te sve raed apa ha
stated a case agaist you aa wrest
ras Sen town ow Stacie te
tna hats nt elo seve youn
cary way. Tree, is ote vn
by svrsenentdrecing you to mae
scence tis Tbr 17-0202 a
roa0aM.

ake rte at incase of you aur
ssa ent stove menting cy belo
in Tuna he cae wba herd ad
xccnnyor aber,

Dee to ongoing Pandemic stuatn, a
tv matters wil be taken up tough
ideo. Conferencing and. for tha

purpose
Al the Advocats/Liigants. sal
fanwrics he Caco Wabexaptcaten
omer
|) Meeting and Password ro re
set nerng cs case oe he
RegstraRecovery Ofcetand Racor
ocr sa be arate oe day rer
wax it x ORT Ofc Pa Le,
argon ncerne Patino Hes.
any exerci tt ba Arca
gas an cara the crcamed ical
2h Wo. 238463
[ies under ry and ae sel of ti
eurlonmste Osa. 2022
by rer oft Tuna
‘ORT Dati
''')

text=[]
label = []

for ent in july_7_2022.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_7_2022= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_7_2022

,Text,Label
0,Debts Recovery Tribunal,TRIBUNAL
1,Jeevan Tra Bulg,ORG
2,Parma Sra,ORG
3,10001,DATE
4,Debts Ove to,TRIBUNAL
5,1983,DATE
6,OA Wo,GPE
7,PUNJAB NATIONAL BANK,ORG
8,Way Prana,GPE
9,Ro 28909,GPE


In [ ]:
july_20_c= nlp('''
DEBTS RECOVERY TRIBUNAL-|, DELHI

4° Floor, Jeovan Tara Building Parlament Street, New Dethi-110001

0A. ne. 60272020
Penh anal Bank (sa Bak ot Commerce) Appian
VER
seittaan Sing Ae Detendaats
ig seaansnoianonne
SiR Ra SWAROOP, RO. 281-C OOA FLATS, GAAZPUR OARY FAR

Sctitom mes oc oa
(@ Misses unocon Pvt.
17a) GAL NO, 1 GOW PARK SAMOARA DEL 10RD, THROUGH TS
recto Se PeaToSnR Cw
ont COPRORATE OFPOEAT W3 SECTOR4S NODA UTTAR PRADESH IE
tino Be NER MOE GC. SECTOR. GREATER NODA WEST. GAT
Blohacan UARPRIOESN OIE enn
er be tve ares iar sted acase crn Ra. 8S

coma Ropar oo 7 on302 2703 AN freer Se dy vat OR mee
[rewsttara grin fre havea 127047)

fev tOAT Onc Pot "ot gout une be Pio Pas
Granny salen at dy
''')

text=[]
label = []

for ent in july_20_c.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_20_c= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_20_c

,Text,Label
0,DEBTS RECOVERY TRIBUNAL-|,TRIBUNAL
1,4,MONEY
2,Jeovan Tara Building Parlament Street,GPE
3,New Dethi-110001,GPE
4,Sing Ae Detendaats,ORG
5,Ra SWAROOP,PERSON
6,281,MONEY
7,GAAZPUR OARY,GPE
8,17a,DATE
9,GAL NO,ORG


In [ ]:
july_20_d = nlp('''
Pad BEFORE DEBTS RECOVERY TRIBUNALMIIl, DELHI

FLOOR, JEEVAN TARA BUILDING, PARLIAMENT STREET, NEW DELI-11000

[0-1] MR RAJESH KUMAR SIO MR HARINAND SHARMA
HNO. 1608, MARUTI KUNG, BHONOS! (188), GURGAON, HARYANA.
122103,

ALSOAT.

MIR. RAJESH KUMAR SiO HARI NANO SHARMA

DESIGNATION MARUTIASSOCIATE

DEPARTMENT - MANCHINE MIC-3

C/O MARUTI SUZUKIINDIALTD.

PLOTINO, 18, PALAM ROAD, UDYOG VIHAR, GURGAON, HARYANA,
122015

MS. SWATI SHARMA
D/O MR VUAY KUMAR SHARMA

HNO. 1608, MARUTI KUNG, BHONOS! (168), GURGAON, HARYANA.
122103,

MSR PROVIEWREAL TECH PYTLTD BUILDER)

8-66, SECTOR-63, NOIDA, DISTRICT GAUTAMBUDH NAGAR, U.P
‘ALSOAT--M/SM.R. PROVIEW REAL TECH PVTLTD (BUILDER)

‘09, OPPOSITE AWAS VIKAS, BHOPURA TILLA MORE, GHAZIABAD,
ue.

Wareas fe above named applicant has nshited a casa opanal you and whereas
nas teen shown Pie salatactn othe Tibunal Matto possBieo serve youn
ordinary way Therefore, ts nce ven by adverssement drecirg yau lo make
sppoance inthis Tvunal 27.07 2022 10.30 AM.

Take notice that incase of your fal fo appear on te above mensoned cay

‘ror fe nex dato at ORT Ofit Porta. dt gov in under re Pubic Notice
Head
(wy vary exgency qua that be Advocate Aarts can contact the concemed
fsa at Ph Mo 23748469,
Given undermyhand and seal othe Tebunal ont he 16° day of June 2022
By Order of the Tribunal
''')

text=[]
label = []

for ent in july_20_d.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_20_d= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_20_d

,Text,Label
0,Pad BEFORE DEBTS RECOVERY TRIBUNALMIIl,ORG
1,DELHI\n\nFLOOR,GPE
2,JEEVAN TARA BUILDING,GPE
3,NEW DELI-11000,GPE
4,HARINAND SHARMA,GPE
5,MARUTI KUNG,ORG
6,188,TIME
7,RAJESH KUMAR SiO HARI NANO SHARMA\n\nDESIGNATI...,ORG
8,MARUTI SUZUKIINDIALTD,ORG
9,18,DATE


In [ ]:
july_21_c = nlp('''
DEBTS RECOVERY TRIBUNAL-|, DELHI

4° Floor, Jeovan Tara Building Parlament Street, New Dethi-110001

0A. ne. 60272020
Penh anal Bank (sa Bak ot Commerce) Appian
VER
seittaan Sing Ae Detendaats
ig seaansnoianonne
SiR Ra SWAROOP, RO. 281-C OOA FLATS, GAAZPUR OARY FAR

Sctitom mes oc oa
(@ Misses unocon Pvt.
17a) GAL NO, 1 GOW PARK SAMOARA DEL 10RD, THROUGH TS
recto Se PeaToSnR Cw
ont COPRORATE OFPOEAT W3 SECTOR4S NODA UTTAR PRADESH IE
tino Be NER MOE GC. SECTOR. GREATER NODA WEST. GAT
Blohacan UARPRIOESN OIE enn
er be tve ares iar sted acase crn Ra. 8S

coma Ropar oo 7 on302 2703 AN freer Se dy vat OR mee
[rewsttara grin fre havea 127047)

fev tOAT Onc Pot "ot gout une be Pio Pas
Granny salen at dy
''')

text=[]
label = []

for ent in july_21_c.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_21_c= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_21_c

,Text,Label
0,DEBTS RECOVERY TRIBUNAL-|,TRIBUNAL
1,4,MONEY
2,Jeovan Tara Building Parlament Street,GPE
3,New Dethi-110001,GPE
4,Sing Ae Detendaats,ORG
5,Ra SWAROOP,PERSON
6,281,MONEY
7,GAAZPUR OARY,GPE
8,17a,DATE
9,GAL NO,ORG


In [ ]:
july_21_d = nlp('''
DEBTS RECOVERY TRIBUNAL - I, DELHI

ty Floor, Joov

JOANo. 27072010
‘State Bank oftndia yg Applicant

ra, Sh: Pawan Tyagh 8 Av Detondants

t> sh. Pawan Tyagl 0 Sh Ninos Tyg. 8 Proprietor of: Ms Mon Electr
Plating, Wate Rood Gal No.6 Shen Roos. Hn Vin Oppose

Re 2447 929 (Rupees Twenty Fur Lacs Forty Seven Town
Keates Remar: ravers
raters baa ior
mreoranitereaes erences
avis meas an anew naman
ete teres waren mene

ere Pte Koen es
‘ein ary exigency aun Mate Advocates / Ligants can contact ‘nl

it ‘or tater
hn Rei ORT Nw Dot Phone o.'oT7e047
''')

text=[]
label = []

for ent in july_21_d.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_21_d= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_21_d

,Text,Label
0,DEBTS RECOVERY TRIBUNAL,TRIBUNAL
1,State Bank,ORG
2,8 Proprietor,TRIBUNAL
3,Ms Mon Electr,ORG
4,Hn Vin Oppose\n\nRe 2447,ORG
5,Rupees Twenty Fur Lacs,ORG
6,Forty Seven Town,MONEY
7,Keates Remar,GPE
8,Pte Koen,GPE
9,Rei ORT Nw Dot Phone,TRIBUNAL


In [ ]:
july_22_a=nlp('''
BEFORE DEBTS RECOVERY TRIBUNAL-III, DELHI

C evan Tara Bu ment Street, New Delhi-110004
‘Notice under Section 19 (4) of fe Recovery of Debts Due to Banks and Financial

Insttatons Act. 1993 rad wih Rue 128 13. the Debts Recovery Tribunal (Procedure
Rides) 193).

Inthe matter of
OA No, 77372021

IDFCFIRSTBANK LTD. Vs_—_LARK LOGISTICS PRIVATE LIMITED AND ORS
Applicant Defendants,

pi | LARK LOGIsTi¢s PVTLTD.
FLATNO.A:227, GROUND FLOOR, DOUBLE STOREY,

ALKALI, NEW DELHI-19

‘ALSO AT: MEZZANINE $5 COMMUNITY CENTRE,

EAST OF KAILASH, DELH»-110019

‘ALSO AT: C42, SECOND FLOOR, BLOCK-C SOUTH EXTENSION,

[PARTI GAUTAM NAGAR, ANDREWSGANJ, SOUTH DELHI,

DEFENCE COLONY, DELHI-110049

(02 _| SHARAD CHANDRA GOEL
FLAT NO. A227, GROUND FLOOR, DOUBLE STOREY,

ALKALI, NEW DELHI-19

‘ALSO AT: MEZZANINE $5 COMMUNITY CENTRE,

EAST OF KAILASH, DELH»-110019

‘ALSO AT: C42, SECOND FLOOR, BLOCK-C SOUTH EXTENSION,
PART.l, GAUTAM NAGAR, ANOREWSGANJ, SOUTH DELHI,
DEFENCE COLONY, DELHI-110040

03 | VIKRAM GOEL
FLAT NO. A227, GROUND FLOOR, DOUBLE STOREY,

KALXAJ NEW DELAM 19

‘ALSO AT: MEZZANINE 85 COMMUNITY CENTRE,

EAST OF KAILASH, DELMI-110019

‘ALSO AT: C-42. SECOND FLOOR, BLOCK:C SOUTH EXTENSION,
PARTI, GAUTAM NAGAR, ANDREWSGANJ, SOUTH DELHI
DEFENCE COLONY, DELH110049

Were the above rarred applicant has instituted a case against you and whereasithas|
been shown the satstacton ofthe Trunal that tot possible fo serve youln ordinary
way. There, his notice i ven by advertsementdectng you lo make appearance in
hs Tribunal 23.08.2022 a 10.30 AM.
Take notice Pat in case of you ature to appear onthe above mentioned day beer Bis |
Tribu case wil be heard and decided m yout absence
Due to ongoing Pandemic stuason, al the matters wil be taken up tough Video|
| Conteracing anor at purpoae
() Al te AdvocatesLtgants shall download the Cisco Webex appiication /sotwar
() Meeting IO and Password forthe next date of hearing qua cases to be taken by |
RegstrarRecovery Ofcarliand Recovery ice. shal be avaiable one day prior
tothe nex! date at ORT Offical Portal. dt gov.n under the PubsicNotie Head
() in any exigency qu at, the Advocates/iSgats can contact the concamed ofc t|
Ph No, 23748469
Given under my handand seach Trbunalonthis the 8th day of July 2022.

By Order ofthe Tribunal
''')

text=[]
label = []

for ent in july_22_a.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_22_a= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_22_a

,Text,Label
0,BEFORE DEBTS RECOVERY TRIBUNAL-III,ORG
1,Tara Bu,ORG
2,New Delhi-110004,GPE
3,Section 19,GPE
4,Debts Due to,TRIBUNAL
5,Banks and Financial\n\nInsttatons,TRIBUNAL
6,1993,DATE
7,13,DATE
8,Debts Recovery Tribunal,TRIBUNAL
9,OA No,ORG


In [ ]:
july_22_b=nlp('''
BEFORE DEBTS RECOVERY TRIBUNAL-II!, DELHI

or Jeevan Tara Building jw Delhitt
‘Notice under Secton 19 (4) ofthe Recovery of Debts Due to Banks and Financial
Insitutons Act 1983 ead wih Rule 128 13 fhe Debts Recovery Tribunal (Procedure
Rides) 1983)

Inthe matter of
(OA. No, 77472021
IDFCFIRSTBANKLTD, Vs —_RADIX PROJECTS PRIVATE LTD. AND ORS
Applicant Defendants
RADIX PROJECTS PVT LTO
Dt | 11401, GL. EDEN HEIGHT. SECTOR-70, GURGAON-122t01
2 | RAGIN SINGH

|H401,GPL, EDEN HEIGHT, SECTOR-TO, GURGAON-122101

3 | SATISH KUMAR

|H401,GPL, EDEN HEIGHT, SECTOR-TO, GURGAON-122101

Whereas the above named applicanthas insisted a case agains you and whereas thas

been shown tothe satistacton ofthe Trbunal hat ts not posse to serve youin ordinary

way. Therefore, his notice is given by advertisement recéng you tomake appearance

hs Tribunal 2,.08-2022 at 10:30AM.

Take notice tat case of your ature to appear onthe above mentioned day bere hs

‘Troural the case wate heardanddecied in your absence

Due to ongoing Pandemic situation, al the matirs wil be taken up trough Video

Conferencing and fr that purpose:

() Al te AdvocatesLigants shall download the Cisco Webex application ‘sofware.

(i) Meeting IO and Password forthe next date of hearing quo cases to be taken by
RegisraRecovery Offcarvand Recovery Oicor hal be avaiable one day prior
toe next date at RT Ofc Portal, dt gov in underthe Public Notice Head

i) many exigency qua at. the Advocates igants can contact he concerned oii a
Ph. No, 23748469

‘Given under my hand and seal ots Tbunalonthis the 8th day o July 2022.

By Order ofthe Tribunal
''')
text=[]
label = []

for ent in july_22_b.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_22_b= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_22_b

,Text,Label
0,BEFORE DEBTS RECOVERY TRIBUNAL-II,ORG
1,Jeevan Tara Building,GPE
2,Delhitt\n‘Notice,ORG
3,Debts Due to,TRIBUNAL
4,Banks and Financial\nInsitutons,TRIBUNAL
5,Debts Recovery Tribunal,TRIBUNAL
6,77472021,DATE
7,Applicant Defendants\nRADIX PROJECTS PVT LTO,ORG
8,EDEN HEIGHT,GPE
9,RAGIN SINGH,GPE


In [ ]:
july_30_b=nlp('''
Debts Recovery Tribunal, Delhi’
Ath lor, Jeevan Tara Bung,
Parmer Stree. New Dei 11001

‘OA. No, 58672020
PUNJAB NATIONAL BANK
Versus Aliant
‘Sh. Pradeep Kaushik & Ors.
Detencant

te
1, Sh. Pradeep Koushik $io_ 8h
Ramlshan Kaushik tHe. 9225
Bock, ‘Sangam Var, Dest Suh
Dem fone Deteawe!
2. WsMLR.DD. Developers Pvt. Lid Ni
58, Moun Road, Oppste dan Tite
Factory Ghana UP 201010
Dena?
Whereas te above nara ages as
nstied a case or reco of Rs.
25,50,24.0 (Rs. Twety Nine Labh Fy |
Thousand Two Hundred Forty Thre
Oni gnestyouard eres thas been
stoun ote sastacton ihe Treat at
K's rk posse i serve you in ria
oy, There fis noton i ven by
acrerisenent decing you te mate
soperance belo Us. 'Regsar on
1800-2022 at 1030 AM. (or futher
details kindly visit DRT website
wustbunlgovin) Phone No. et
zea)
Toke notice that in case of your tae
appear ore above merennes dey be
is Thbual, the case wil be head an
ecsdtinyor bear.
Due to ongoing Pandemic Situation, a
the matters wil be taken up tough |
Video Conferencing and for that

purpose:-
(i) All the Advocates1Liigants shal
emote "Cac Weber pcan
i) “Weetng 10° and “Passwort” fr te
nex date of hearing qua cases tobe ink |
by RegistalRecovery Oricers! and
covery Ofc shal be avatate oe |
ay rir to tho net date at ORT Ofc
Portal ie. “értgovin" under the Pubic
Notice Heas
(i) In any exigency qua that, the!
[AdvecatesUiigants can. contact the!
concemed official at Ph. No. Ot
2514473,
[Given under my hand and seat of the!
‘rounalon Bis 20° June of 2022.

‘by order of hs Tbunal

‘Assistant Registrar

Respondent may contact under meson
rene runertor ster enqury.
Art K. as, Ls. Regitar, ORTA, New
Dati Pron: o-23748473
E-malt dtidebi-dfs@nicin
''')

text=[]
label = []

for ent in july_30_b.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_30_b= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_30_b

,Text,Label
0,Debts Recovery Tribunal,TRIBUNAL
1,Delhi’,GPE
2,Jeevan Tara Bung,GPE
3,Parmer Stree,GPE
4,New Dei 11001,GPE
5,58672020,DATE
6,PUNJAB NATIONAL BANK,ORG
7,Pradeep Kaushik & Ors,ORG
8,Pradeep Koushik,ORG
9,Ramlshan Kaushik,GPE


In [ ]:
july_31=nlp('''
BEFORE DEBTS RECOVERY TRIBUNAL -Ill, DELHI

Jeevan Tara Building, Parliament Street, New Delhi-110001

Notice under Secton 14) ol he Recovery of Debts Ove to Banks and Fanci isttutons
Act 1983 read wit Rule 12 13ol the Debts Recovery Trounal Procedure Rules) 1988

Rajesh Kumar and Others]
Defendants
Kumar and Others, So Ms Hai Nand Sharma, No. 1608, Man Kun
tonal, pon Naor 12005 Algo AE SM, Hat Nand Sear,
‘ara essa, bepartent- Nacho he Go ana Suna
Vins, Haryana, Gurgaon: 122015
‘Vjay Kamar Sharma, H No, 1608, Mor Kun), Bhonds
ra, Garg 122109
MR Proview Reahech Pvt Lid. Tough is Orecr, B 8, See 8. Noa
Distinct Gavtam Budh Nagar, UP, Also At 99 Opp, Avs Vas, Bhopura Tila Mere,
IGrazied, UP.
ivereas he above named aplicanthas nstized a case aganst you ana whereas thas
Peer shown 2a tstatcn of ne Tita! alt toni sae yun Be
ary way Thor. ae gen averse rein you b ma
speartce ths Trou! 16.09.2022 a 10:30 m
[ate notce Pat case of you ano to appear onthe above mentones cay betere th]
rou case wil be heard andoeciad your absence
ove fo angoing PaneacStuston al th mates wil be taken up tough
2nd for that purpose) Al pe AdvocalasLtgarts shal downicat De}
cisco Weber” aplcaton. sotwae (i) Ween I” ana Passwore fo he rx! date
''')

text=[]
label = []

for ent in july_31.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_july_31= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_july_31

,Text,Label
0,BEFORE DEBTS RECOVERY TRIBUNAL,ORG
1,DELHI\n\n,GPE
2,Jeevan Tara Building,GPE
3,Parliament Street,GPE
4,New Delhi-110001,GPE
5,Debts Ove to,TRIBUNAL
6,Act 1983,DATE
7,Rule 12,MONEY
8,Debts Recovery Trounal Procedure Rules,TRIBUNAL
9,Ms Hai Nand Sharma,PERSON


In [ ]:
bs_aug1=nlp('''
FORM NO. [See Regulation 3(2]
(OFFICE OF THE RECOVERY OFFICER:
DEBTS RECOVERY TRIBUNAL OELMIORT
4 Foo Jeevan Tra Bldg, Parlanent Steet, New eh 000%
Notice Setting Sale reclamation Under Rule othe Second Sheds iote|
Income Taxa, 161 Rea withthe RecovryofDabs 8 Bankrupt Ac 1380
Rano DBI BANK LTD. ‘27202

Versus
ro, 'WS HOLYSTAR INFRASTRUCTURE PVT.LTD.

(co 1) ms Holstarintastuctue Prt. Lid, 68-10, Bock, Ground Fer
Cenzu Mat Kass No 389 Ma MG. Rood Vilage Suanpur Osh 90 Als at
3638 Himeyunur, StéyngEndave, New Debi
(c02) nPardeepPaliwal-1138.0ck Vesa Var New Osh
(c03) sh. VinayakBhat, £138 Bock Vasant iba New Deh Aloat:C-1 ng
For GreenPart NewDehi
eres you ews ceed he Presa Ow o DEBTS RECOVERY TRGUNAL|
DEL ORT wo nacisuedthe Recrven Case sac 05072019 O50
io pay wear: aris)Frarcilszen() Name apocas, bo sum of
za785801281 (wen Eg Coes Seeny Sx Lats Nn Ei Those
ave on Pase Egy Gre On srg wth penenetleanure rites owe
it reason and eas of 0 (On). an whereas esa has ot boat ple
unersgnodnas creo sact uceroriond inex! Inevesiopopa)
> Youare heey rome that te 108202 at 1220 AM na bee fed ring
ote grcamatonotslsnd soi holes tron Yvareogsteso berg ho
rice tbe ndersonedny enarbrance, cages, cms clases sachet he
sid pepanesoranyporentbrt
Specteaton of Property
emote but up prepay anaes of 2570 o,f Equly did over up|
tosonert, est second ar tid ous and invite popuronso sure na
reasurg 1 Bas shales Ora of Wage Surg Tet Hae Ks, New
Den temngpatt Conical Comply, Conran
Guenundermynandante seat Tal oni cat oan7z2
Recovery Ofc
‘Debts Recovery Tribunal Dei (ORT.
''')

text=[]
label = []

for ent in bs_aug1.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_bs_aug1= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_bs_aug1

,Text,Label
0,See Regulation,GPE
1,OFFICE OF THE RECOVERY OFFICER,TRIBUNAL
2,DEBTS RECOVERY TRIBUNAL,TRIBUNAL
3,4 Foo Jeevan Tra Bldg,MONEY
4,Parlanent Steet,GPE
5,000%\n,TRIBUNAL
6,Notice Setting Sale,ORG
7,Second Sheds,GPE
8,161,MONEY
9,Bankrupt Ac,GPE


In [ ]:
bs_aug4=nlp('''
‘FORM NO.
‘DEBTS RECOVERY TRBUNAL. LUCKNOW
60041, ery Road Near hanuman Se emee Licino 22007,
(Wael Jatin Patt Utr Pech)
(Seon fg Rp & ponarance by putin)
 surenns teat under Sse 8) the Reco Debs ue Bes 2d
anda intaor e185 Rute 12a 3th Des Recovery tual
Proce Rls,185),
‘ORIGINAL APPLICATION No. 872 of 2020,
ese Reconstruction Company ) Lined Aeptant
VERSUS.

1 ws be Ge Scns an Techno Lint, compsy incor unde te
Cong ct 128 hing repre ces Cl Lie, stds Pu
Post Cargrale fe oP 8; Sac, Gurgeoa, Hanna Anat
‘OF Noda Spi zonone Zr, Noida Dat Roa, Noa Aaa Pt 3
(18, Nada Spel esr Zone ae Nia Dash Ros Naa.

> Depa somber ot Sve Pra Loom, aged abouts ears, est
Fs, Rach Apament, Hah Va Parpua shi and abo a 124 (A
Pensa Presi Sar 92, Nota.

sts: Ha ne Leonie Ocak Lona, pe about 4S yas sen
#6124 (A)PaasmathPresige, Sece34 Noida

4 D61Bark Ue, ian Rod Cos oct Shaan, Red Coss Mar, Now De
‘000

ne stove notes apcaton, you ae equtedt tel nPapec Book tami To

ng wi documents se tds i any) personaly or aug our dl aubased

gerry praconern ts Teun, ate erg copy obese cn ta sppae

i coneally arenes aon afer aig coy ho sane en be spler ahs

crcl aubotza get ater pbieaton be surmare rl heeaer bape

etree Tra 808.222 1030 AML, leg whcn be apleon sal
rearsanddecdesin yar aber
''')

text=[]
label = []

for ent in bs_aug4.ents:
  text.append(ent.text)
  label.append(ent.label_)
df_bs_aug4= pd.DataFrame(zip(text,label), columns = ['Text','Label'])
df_bs_aug4

,Text,Label
0,DEBTS RECOVERY TRBUNAL,TRIBUNAL
1,Wael Jatin Patt Utr Pech,ORG
2,Rp &,ORG
3,Sse 8),MONEY
4,12a 3th Des Recovery,MONEY
5,872,MONEY
6,Reconstruction Company,ORG
7,Ge Scns an Techno Lint,ORG
8,Cl Lie,GPE
9,Pu\nPost Cargrale,ORG


In [ ]:
dfs  = {'df_dec_3_2021':df_dec_3_2021, 'df_dec_8_2021':df_dec_8_2021, 'df_may_20_2022':df_may_20_2022, 'df_july_7_2022':df_july_7_2022,
        'df_july_20_c' : df_july_20_c, 'df_july_20_d' :df_july_20_d, 'df_july_21_c' :df_july_21_c , 'df_july_21_d' :df_july_21_d, 'df_july_22_a' : df_july_22_a, 
        'df_july_22_b' :df_july_22_b, 'df_july_30_b' : df_july_30_b, 'df_july_31' :df_july_31, 'df_bs_aug1' : df_bs_aug1, 'df_bs_aug4' :df_bs_aug4 }
writer = pd.ExcelWriter('New_Output_DRT_1.xlsx') 

for name,dataframe in dfs.items():
    dataframe.to_excel(writer,name,index=False)

writer.save()

In [ ]:
''''
writer = pd.ExcelWriter('DRT_Output.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('NER')
writer.sheets['NER'] = worksheet
df_july_20_c.to_excel(writer,sheet_name='Validation',startrow=1 , startcol=0)   
df_july_20_d.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_20_c.shape[1]+2) 
df_july_21_c.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_20_d.shape[1]+2) 
df_july_21_d.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_21_c.shape[1]+2) 
df_july_22_a.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_21_d.shape[1]+2) 
df_july_22_b.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_22_a.shape[1]+2) 
df_july_30_b.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_22_b.shape[1]+2) 
df_july_31.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_30_b.shape[1]+2) 
df_bs_aug1.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_july_31.shape[1]+2) 
df_bs_aug4.to_excel(writer,sheet_name='Validation',startrow=1, startcol=df_bs_aug1.shape[1]+2) 
''''

SyntaxError: ignored